<a href="https://colab.research.google.com/github/sunyeul/ToyProjectLab/blob/feature%2Fstable_diffusion_and_controlnet_webui/Stable_Diffusion_WebUI/Stable_Diffusion_ControlNet_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon Jun 12 09:00:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchtext torchaudio torchdata==0.6.0 --index-url https://download.pytorch.org/whl/cu118 -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 636.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 100.8 MB/s eta 0:00:00


In [ ]:
# webuiをインストール
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

Cloning into 'stable-diffusion-webui'...
remote: Enumerating objects: 22369, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 22369 (delta 27), reused 38 (delta 16), pack-reused 22311
Receiving objects: 100% (22369/22369), 30.47 MiB | 38.37 MiB/s, done.
Resolving deltas: 100% (15598/15598), done.


In [ ]:
# sd-webui-controlnet のダウンロード
!git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet

Cloning into '/content/stable-diffusion-webui/extensions/sd-webui-controlnet'...
remote: Enumerating objects: 7212, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 7212 (delta 66), reused 87 (delta 48), pack-reused 7099
Receiving objects: 100% (7212/7212), 15.59 MiB | 21.87 MiB/s, done.
Resolving deltas: 100% (4050/4050), done.


In [ ]:
# 作業ディレクトリを移動させておく
%cd /content/stable-diffusion-webui

/content/stable-diffusion-webui


In [ ]:
# SDモデルのダウンロード
!wget https://huggingface.co/nuigurumi/basil_mix/resolve/main/Basil_mix_fixed.safetensors -O /content/stable-diffusion-webui/models/Stable-diffusion/Basil_mix_fixed.safetensors
!wget https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_6_BakedVae.safetensors -O /content/stable-diffusion-webui/models/Stable-diffusion/DreamShaper_6_BakedVae.safetensors
!wget https://huggingface.co/sazyou-roukaku/chilled_remix/resolve/main/chilled_remix_v2.safetensors -O /content/stable-diffusion-webui/models/Stable-diffusion/chilled_remix_v2.safetensors

# VAEダウンロード（ここは固定）
!wget https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -O /content/stable-diffusion-webui/models/VAE/vae-ft-mse-840000-ema-pruned.safetensors

--2023-06-12 09:02:01--  https://huggingface.co/nuigurumi/basil_mix/resolve/main/Basil_mix_fixed.safetensors
Resolving huggingface.co (huggingface.co)... 18.67.0.55, 18.67.0.90, 18.67.0.67, ...
Connecting to huggingface.co (huggingface.co)|18.67.0.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/5b/ec/5bec02f48cde4dc46a4a3590ccf377e5b59a146f5f769f4291923392aaa8315a/0ff127093f5be455057742c40cef578407b6933f240ee8dc5ed0f3061196fb38?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27Basil_mix_fixed.safetensors%3B+filename%3D%22Basil_mix_fixed.safetensors%22%3B&Expires=1686819722&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzViL2VjLzViZWMwMmY0OGNkZTRkYzQ2YTRhMzU5MGNjZjM3N2U1YjU5YTE0NmY1Zjc2OWY0MjkxOTIzMzkyYWFhODMxNWEvMGZmMTI3MDkzZjViZTQ1NTA1Nzc0MmM0MGNlZjU3ODQwN2I2OTMzZjI0MGVlOGRjNWVkMGYzMDYxMTk2ZmIzOD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iOn

In [ ]:
# ControlNetモデルのダウンロード
!wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_v11f1e_sd15_tile.pth
!wget https://huggingface.co/ioclab/ioc-controlnet/resolve/main/models/control_v1p_sd15_brightness.safetensors -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_v1p_sd15_brightness.safetensors

--2023-06-12 09:03:09--  https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth
Resolving huggingface.co (huggingface.co)... 18.67.0.90, 18.67.0.55, 18.67.0.34, ...
Connecting to huggingface.co (huggingface.co)|18.67.0.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/e7/4c/e74c9ddc88b5a293fa8aafeb615d6c61cb3aaa34dc7ce9e25100dfae2b09bf55/8aa69b8d391e72c2fced6a650268137dc0ed594cafe8a2c0f8b994799a21979b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27control_v11f1e_sd15_tile.pth%3B+filename%3D%22control_v11f1e_sd15_tile.pth%22%3B&Expires=1686819790&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2U3LzRjL2U3NGM5ZGRjODhiNWEyOTNmYThhYWZlYjYxNWQ2YzYxY2IzYWFhMzRkYzdjZTllMjUxMDBkZmFlMmIwOWJmNTUvOGFhNjliOGQzOTFlNzJjMmZjZWQ2YTY1MDI2ODEzN2RjMGVkNTk0Y2FmZThhMmMwZjhiOTk0Nzk5YTIxOTc5Yj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25k

In [ ]:
# webuiを別URLで立ち上げる（少し時間がかかる）
!python launch.py --share --xformers --enable-insecure-extension-access

Python 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]
Version: v1.3.2
Commit hash: baf6946e06249c5af9851c60171692c44ef633e0
Installing gfpgan
Installing clip
Installing open_clip
Installing xformers
Cloning Stable Diffusion into /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai...
Cloning Taming Transformers into /content/stable-diffusion-webui/repositories/taming-transformers...
Cloning K-diffusion into /content/stable-diffusion-webui/repositories/k-diffusion...
Cloning CodeFormer into /content/stable-diffusion-webui/repositories/CodeFormer...
Cloning BLIP into /content/stable-diffusion-webui/repositories/BLIP...
Installing requirements for CodeFormer
Installing requirements
Installing sd-webui-controlnet requirement: mediapipe
Installing sd-webui-controlnet requirement: svglib
Installing sd-webui-controlnet requirement: fvcore

Launching Web UI with arguments: --share --xformers --enable-insecure-extension-access
2023-06-12 09:05:32.482458: I tensorflow/co